# encoding

> We define all the usefull functions to encode the TR sequence as an imput for the predictive model

In [ ]:
#| default_exp encoding

In [ ]:
# | export
import ViennaRNA as RNA
import numpy as np
from tqdm import tqdm


In [ ]:
#| export

def base_pairing_probability_array(sequence: str, #DNA sequence
                              ):
                              
    """Calculates the probability pairing array for a given DNA sequence"""

    # Convert DNA sequence to RNA (replace T with U)
    rna_seq = sequence.replace('T', 'U')

    # Perform RNA folding using ViennaRNA library
    (propensity, ensemble_energy) = RNA.pf_fold(rna_seq)

    # Create a square array with dimensions equal to the sequence length
    size = len(sequence)
    array = np.zeros((size, size), dtype=np.float16)  # Empty array for probabilities

    # Fill the array with pairing probabilities using RNA.get_pr()
    for i in range(size):
        for j in range(size):
            array[i, j] = RNA.get_pr(i + 1, j + 1) 

    # Round probabilities to 4 decimal places
    return np.round(array, decimals=4)


In [ ]:
#| hide
bppa=base_pairing_probability_array('ATCCCGGCAGC')
np.all(bppa==np.array([[0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.0001, 0.0002, 0.    ,
        0.0003, 0.0001, 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.0004, 0.    ,
        0.    , 0.0017, 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.0015, 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.0144, 0.    ],
       [0.    , 0.0001, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.0064],
       [0.    , 0.0002, 0.0004, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.0002],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    ],
       [0.    , 0.0003, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    ],
       [0.    , 0.0001, 0.0017, 0.0015, 0.0144, 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.0064, 0.0002, 0.    ,
        0.    , 0.    , 0.    ]], dtype=np.float16))

True

In [ ]:
#| export
def base_pairing_probability_max_mean_std(sequence: str, #DNA sequence
                                       ):
    """
    Calculates summary statistics for the base pair probabilities of an DNA sequence.
    """

    # Calculate the probability pairing array using the provided function
    proba_array = base_pairing_probability_array(sequence)

    # Check if the dimensions of the pairing array and sequence match
    size_1 = len(proba_array)
    size_2 = len(sequence)
    if size_1 != size_2:
        raise ValueError("Dimensions of probability array and sequence mismatch!")

    # Create a new array to store summary statistics
    proba_array_summary = np.zeros((size_1, 3))

    # Calculate and store maximum probability for each position
    for k in range(size_1):
        proba_array_summary[k, 0] = np.max(proba_array[k])

    # Calculate and store mean probability for each position
        proba_array_summary[k, 1] = np.mean(proba_array[k])

    # Calculate and store standard deviation for each position
        proba_array_summary[k, 2] = np.std(proba_array[k])

    # Return the array of summary statistics
    return np.round(proba_array_summary,decimals=10)


In [ ]:
#| hide
bppmms=base_pairing_probability_max_mean_std('ATCCCGGCAGC')
np.all(bppmms==np.array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.99930600e-04, 6.36578000e-05, 0.00000000e+00],
       [1.70040130e-03, 1.90973300e-04, 4.88281200e-04],
       [1.50012970e-03, 1.36375400e-04, 4.22954600e-04],
       [1.43966675e-02, 1.30844120e-03, 4.14276120e-03],
       [6.40106200e-03, 5.90801200e-04, 1.84345250e-03],
       [4.00066400e-04, 7.27177000e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.99930600e-04, 2.72393000e-05, 0.00000000e+00],
       [1.43966675e-02, 1.60884860e-03, 4.08554080e-03],
       [6.40106200e-03, 5.99861100e-04, 1.84345250e-03]]))

True

In [ ]:
# |export
def encoding_RNA_pairing(sequence_list:str #list of DNA sequences of TRs
                                        ):
    """
    Encodes a list DNA sequences based on base pair probability statistics.
    The DNA sequence here correspond to  20 bases of Avd + the TR sequence + 50/100/150 bases of the DGR spacer
    The encoding is independent of the TR length
    """

    encoding_list = []  # List to store encoded sequences
    Avd_seq='aagggcaggctgggaaATAA'.upper()
    DGR_spacer='tctgtgcccatcaccttcttgcatggctctgccaacgctacggcttggcgggctggcctttcctcaataggtggtcagccggttctgtcctgcttcggcgaacacgttacacggttcggcaaaacgtcgattactgaaaatggaaaggcggggccgacttc'.upper()

    for sequence in tqdm(sequence_list):
        # Define spacer lengths 
        spacer_lengths = [50, 100, 150]

        # Get transcript length
        tr_length = len(sequence)

        # Iterate through spacer lengths
        for k in spacer_lengths:
            # Construct sequence with spacer
            sequence_k = Avd_seq + sequence + DGR_spacer[:k]  # Replace with actual sequences

            # Calculate base pair probability statistics
            bppmms_k = base_pairing_probability_max_mean_std(sequence_k)

            # Extract statistics for Avd region
            bppmms_avd = bppmms_k[:20, :] 

            # Extract statistics for TR region (split into 10 parts)
            bppmms_tr = bppmms_k[20:20 + tr_length, :]
            bppmms_tr_split = np.array_split(bppmms_tr, 10, axis=0)  # Split into 10 parts
            bppmms_tr_mean = np.array([np.mean(k, axis=0) for k in bppmms_tr_split])  # Mean of each split

            # Extract statistics for spacer region
            bppmms_spacer = bppmms_k[20 + tr_length:, :]  # Adjust slice based on spacer region size

            # Merge statistics for different regions
            bppmms_merged = np.concatenate([bppmms_avd, bppmms_tr_mean, bppmms_spacer], axis=0)

            # Concatenate encodings for different spacer lengths (if applicable)
            if k != spacer_lengths[0]:
                bppmms_concat = np.concatenate([bppmms_concat, bppmms_merged], axis=0)
            else:
                bppmms_concat = bppmms_merged

        # Append encoding for the current sequence
        encoding_list.append(bppmms_concat)

    # Convert to numpy array for efficient processing
    return np.array(encoding_list)


In [ ]:
# |hide
emmstsi=encoding_RNA_pairing(['ATCCCGGCAGC'])

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  7.75it/s]


In [ ]:
# |export
def bases_frequency(sequence:str #The DNAsequence as a string
                     ):
    """
    Calculates the normalized frequency of each base in a given sequence.
    """

    # Initialize an array to store base counts
    count_normalized = np.zeros(4)

    # Define a list of bases
    base_list = ['A', 'C', 'G', 'T']

    # Count occurrences of each base in the sequence
    for k in range(len(base_list)):
        count_normalized[k] = sequence.count(base_list[k]) / len(sequence) # Normalize counts to represent frequencies between 0 and 1
    
    return np.round(count_normalized,decimals=4)

In [ ]:
bases_frequency('ATCCCGGCAGC')

array([0.1818, 0.4545, 0.2727, 0.0909])

In [ ]:
# |export
def base_before_after_A(sequence:str #The DNAsequence as a string.
                       ):
    """
    Counts the occurrences of each base before and after each 'A' base in a sequence.
    """

    # Initialize arrays to store counts
    list_before = np.zeros(4, dtype=int)  
    list_after = np.zeros(4, dtype=int)
    base_list = ['A', 'C', 'G', 'T']  
    
    # Iterate through each base in the sequence
    for k in range(len(sequence)):
        # Check if the current base is an 'A'
        if sequence[k] == 'A':
            # Handle edge cases: first and last base
            if k == 0:  # First base is an 'A'
                list_before[0] += 1  # 'A' itself doesn't have a base before
                for j in range(len(base_list)):
                    if k + 1 < len(sequence) and sequence[k + 1] == base_list[j]:
                        list_after[j] += 1
            elif k == len(sequence) - 1:  # Last base is an 'A'
                list_after[3] += 1  # 'A' itself doesn't have a base after
                for j in range(len(base_list)):
                    if k - 1 >= 0 and sequence[k - 1] == base_list[j]:
                        list_before[j] += 1
            else:  # Middle base is an 'A'
                for j in range(len(base_list)):
                    if k - 1 >= 0 and sequence[k - 1] == base_list[j]:
                        list_before[j] += 1
                    if k + 1 < len(sequence) and sequence[k + 1] == base_list[j]:
                        list_after[j] += 1

    # Combine and return the counts as a numpy array
    return np.concatenate([list_before, list_after])

In [ ]:
base_before_after_A('ATCCCGGCAGC')

array([1, 1, 0, 0, 0, 0, 1, 1])

In [ ]:
# |export
def generate_dic_encode_two():
    """
    Creates a dictionary encoding each dinucleotide with a one-hot encoded vector.
    """

    base_list = ['A', 'C', 'G', 'T']  # DNA bases
    dic_encode = {}  # Dictionary to store encodings
    count = 0  # Index for one-hot vectors

    # Iterate through all possible dinucleotide combinations
    for i in range(len(base_list)):
        for j in range(len(base_list)):
            # Create a one-hot encoded vector with 16 elements
            zero_list = [0] * 16

            # Set the element corresponding to the dinucleotide index to 1
            dinucleotide = base_list[i] + base_list[j]
            zero_list[count] = 1

            # Add the dinucleotide and its encoding to the dictionary
            dic_encode[dinucleotide] = zero_list

            # Increment the index for the next encoding
            count += 1

    return dic_encode


In [ ]:
generate_dic_encode_two()

{'AA': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'AC': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'AG': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'AT': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'CA': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'CC': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'CG': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'CT': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 'GA': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'GC': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 'GG': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 'GT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 'TA': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 'TC': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 'TG': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 'TT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

In [ ]:
# |export
def count_base_duo(sequence:str, #The input sequence to analyze
                    base_duo:str #The two-base pattern to search for within the sequence
                    ):
   """
   Counts the number of times a specified base duo (two consecutive bases) appears within a given sequence.
   """

   # Determine the length of the input sequence
   seq_length = len(sequence)

   # Initialize a counter to track the occurrences of the base duo
   count = 0

   # Iterate through the sequence, examining pairs of consecutive bases
   for k in range(seq_length - 1):
       # Check if the current base pair matches the target base duo
       if sequence[k] + sequence[k + 1] == base_duo:
           # If a match is found, increment the counter
           count += 1

   # Return the final count of base duo occurrences
   return count


In [ ]:
for duo in generate_dic_encode_two():
    print (duo, count_base_duo('ATCCCGGCAGC',duo))

AA 0
AC 0
AG 1
AT 1
CA 1
CC 2
CG 1
CT 0
GA 0
GC 2
GG 1
GT 0
TA 0
TC 1
TG 0
TT 0


In [ ]:
# |export
def two_bases_frequency(sequence:str #The input sequence to analyze
                        ):
    """
    Calculates the relative frequency of each possible base duo (two consecutive bases) within a given sequence.
    """

    # Initialize an empty dictionary to store base duo counts
    count_normalized = {}

    # Define a dictionary to encode base pairs as single characters
    dic_encode = generate_dic_encode_two()  # Assuming this function exists and returns the encoding dictionary

    # Iterate through each possible base duo in the encoding dictionary
    for base_duo in dic_encode:

        # Calculate the count of the current base duo using the count_base_duo function
        count_normalized[base_duo] = count_base_duo(sequence, base_duo) / (len(sequence) - 1)

        # Handle potential errors or edge cases
        if count_normalized[base_duo] < 0:  # Ensure normalized frequencies are non-negative
            raise ValueError("Negative frequency encountered. Check input sequence and encoding dictionary.")

    # Convert the dictionary of normalized frequencies into a NumPy array
    count_normalized = np.array(list(count_normalized.values()))

    # Return the NumPy array containing the base duo frequencies
    return np.round(count_normalized,decimals=4)

In [ ]:
two_bases_frequency('ATCCCGGCAGC')

array([0. , 0. , 0.1, 0.1, 0.1, 0.2, 0.1, 0. , 0. , 0.2, 0.1, 0. , 0. ,
       0.1, 0. , 0. ])

In [ ]:
# |export
def encode_base_by_two(sequence:str # The input DNA sequence to encode.
                       ):
    """
    Encodes a DNA sequence by converting each base duo (two consecutive bases) into a one-hot encoded representation.
    """

    # Define a list of possible DNA bases
    base_list = ['A', 'C', 'G', 'T']

    # Assume a function `generate_dic_encode_two()` exists that returns a dictionary mapping base duos to one-hot encodings
    dic_encode = generate_dic_encode_two()

    # Create a NumPy array with zeros to store the one-hot encodings (shape: sequence length x number of base duos)
    base_encode_duo = np.zeros((len(sequence), 16))

    # Iterate through each position in the sequence (except the last one)
    for k in range(len(sequence) - 1):
        # Extract the current base duo
        base_duo = sequence[k] + sequence[k + 1]

        # Check if the base duo exists in the encoding dictionary
        if base_duo not in dic_encode:
            raise ValueError(f"Invalid base duo encountered: {base_duo}. Check input sequence and encoding dictionary.")

        # Get the one-hot encoding for the base duo and store it in the array
        base_encode_duo[k] = dic_encode[base_duo]

    # Handle the last base in the sequence (no duo available)
    if len(sequence) > 0:
        base_encode_duo[-1] = dic_encode[sequence[-1] + 'T']  # Encode with 'T' as the second base

    # Split the encoded array into 10 chunks for calculating means
    splitted_encoded_duo = np.array_split(base_encode_duo, 10)

    # Calculate the mean of each base duo encoding across each chunk
    mean_splitted_encoded_duo = np.array([np.mean(k, axis=0) for k in splitted_encoded_duo])

    # Flatten the array to obtain a single 1D representation
    return mean_splitted_encoded_duo.flatten()


In [ ]:
encode_base_by_two('ATCCCGGCAGC')

array([0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. ])

In [ ]:
# |export
def encoding_seq_feature(list_TRs: list # A list of DNA sequences (strings) to encode.
                         ):
    """
    Encodes a list of DNA sequences by combining various encoding methods.
    """

    # Initialize an empty list to store the encoded features
    new_features_encoded = []

    # Iterate through each DNA sequence in the list
    for TR in list_TRs:

        # Calculate the base frequency features
        encoded_feature = bases_frequency(TR) 

        # Calculate the base-before-after-A features
        encoded_feature = np.concatenate((encoded_feature, base_before_after_A(TR)))  

        # Encode the sequence using base duo representation
        encoded_feature = np.concatenate((encoded_feature, encode_base_by_two(TR))) 

        # Append the encoded features for this sequence to the list
        new_features_encoded.append(encoded_feature)

    # Convert the list of encoded features into a NumPy array
    return np.array(new_features_encoded)


In [ ]:
# |hide 
encoding_seq_feature(['ATCCCGGCAGC'])

array([[0.1818, 0.4545, 0.2727, 0.0909, 1.    , 1.    , 0.    , 0.    ,
        0.    , 0.    , 1.    , 1.    , 0.    , 0.    , 0.    , 0.5   ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 1.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 1.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.    , 0.    ,
        0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 

In [ ]:
# |export
def encode_tr_list(list_TRs:list ## A list of TRs DNA sequences (strings) to encode
                   ):
    "Concatenate the two encoding (RNA pairing and DNA TR sequence encoding)"
    encode_pairing=encoding_RNA_pairing(list_TRs)
    encode_pairing=encode_pairing.reshape((encode_pairing.shape[0],encode_pairing.shape[1]*encode_pairing.shape[2]))
    encoding_new_features=encoding_seq_feature(list_TRs)
    return np.concatenate((encode_pairing,encoding_new_features),axis=1)